move audio commands separately
to be able to use them in playbook
use playbook to perform voice tests
later can be wrapped into pytest
no need to talk yey!

In [92]:
import pymumble_py3
import numpy as np
import resampy
import time
from voice_forge import PiperTts
from pymumble_py3.constants import PYMUMBLE_SAMPLERATE
from assistant.config import (
    MUMBLE_SERVER_PORT,
    MUMBLE_SERVER_PASSWORD,
    PIPER_MODELS_LOCATION
)

In [93]:
tts = PiperTts("en_US-kathleen-low", PIPER_MODELS_LOCATION)
mumble = pymumble_py3.Mumble(
    host="127.0.0.1",
    user="Query",
    port=MUMBLE_SERVER_PORT,
    password=MUMBLE_SERVER_PASSWORD,
)

mumble.start()

In [94]:
def synthesize_and_stream(mumble: pymumble_py3.Mumble, tts: PiperTts, text: str):
    data, samplerate = tts.synthesize_stream(text)
    resampled_data = resampy.resample(data, samplerate, PYMUMBLE_SAMPLERATE)

    # NOTE: Added 1 second of silence, because VAD needs silence after sentence to trigger.
    esl = int(PYMUMBLE_SAMPLERATE * 1)
    bsl = int(PYMUMBLE_SAMPLERATE * 0.2)
    esl_data = np.zeros(esl, dtype=resampled_data.dtype)
    bsl_data = np.zeros(bsl, dtype=resampled_data.dtype)
    audio_with_silence = np.concatenate((bsl_data, resampled_data, esl_data))

    # Send the data to the sound output
    mumble.sound_output.add_sound(audio_with_silence.astype(np.int16).tobytes())

In [106]:
synthesize_and_stream(mumble, tts, "Why is the sky blue?")

In [123]:
synthesize_and_stream(mumble, tts, "What is COVID-19?")

In [ ]:
synthesize_and_stream(mumble, tts, "Wait.")

In [125]:
synthesize_and_stream(mumble, tts, "What happend?")

In [ ]:
synthesize_and_stream(mumble, tts, "I'm a cat!")

In [47]:
synthesize_and_stream(mumble, tts, "I need food! There no more humans on planet...")

In [ ]:
synthesize_and_stream(mumble, tts, "No! I'm a cat! I need food! There no more humans on planet...")

In [87]:
synthesize_and_stream(mumble, tts, "What is the funniest meme for last 10 years?")

In [88]:
synthesize_and_stream(mumble, tts, "What is your favorite?")

In [98]:
synthesize_and_stream(mumble, tts, "Remember a number 32.")

In [90]:
synthesize_and_stream(mumble, tts, "What is the number I asked to remember?")

In [56]:
synthesize_and_stream(mumble, tts, "Who are you?")

In [28]:
synthesize_and_stream(mumble, tts, "What is you purpose?")